# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import requests
import scipy

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/'

fileNames = ["dom.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

img = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(img, 'gray')
plt.axis('off')
plt.show()


In [ ]:
def crossing(LoG, threshold):
    output_img = np.zeros(LoG.shape)
    height, width = LoG.shape
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            window = LoG[i - 1:i + 2, j - 1:j + 2]
            min_ = np.min(window)
            max_ = np.max(window)
            if min_ * max_ < 0:
                if LoG[i, j] > 0:
                    output_img[i, j] = LoG[i, j] + np.abs(min_)
                if LoG[i, j] < 0:
                    output_img[i, j] = np.abs(LoG[i, j]) + max_
            
    output_img_ = cv2.normalize(output_img, None, 0, 255, cv2.NORM_MINMAX)
    output_img = np.where(output_img_ < threshold, 0, 1)

    output_img = output_img * 255
    output_img = output_img.astype("uint8")

    output_img = cv2.medianBlur(output_img, 5)
    return output_img

In [ ]:
img = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
img = cv2.GaussianBlur(img, (5, 5), 0)
img = cv2.Laplacian(img, cv2.CV_64F)
img_cross = crossing(img, 63)


_, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img, 'gray')
ax[0].axis('off')
ax[1].imshow(img_cross, 'gray')
ax[1].axis('off')

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
def nonmax(quantized_matrix, amplitude_matrix):
    #5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
    # Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
    # Algorytm przebiega następująco.
    # W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    #     - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    #     - przeanalizuje sąsiadów leżących na tym kierunku.
    # Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
    # W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
    # Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
    # Zaimplementuj funkcję `nonmax`.
    # Pierwszym argementem jest macierz kierunków (po kwantyzacji).
    # Drugim argumentem jest macierz amplitudy.
    g_n = np.zeros(quantized_matrix.shape)
    #duplikacja kodu go brrr
    for i in range(1, quantized_matrix.shape[0] - 1):
        for j in range(1, quantized_matrix.shape[1] - 1):
            #     - pionowy ($d_1$),
            if quantized_matrix[i, j] == 0: 
                    if ( amplitude_matrix[i, j] > amplitude_matrix[i, j - 1] and amplitude_matrix[i, j] > amplitude_matrix[i, j + 1]):
                        g_n[i, j] = amplitude_matrix[i, j]
                    else:
                        g_n[i, j] = 0
            #     - poziomy ($d_2$),
            elif quantized_matrix[i, j] == 1:
                    if (amplitude_matrix[i, j] > amplitude_matrix[i - 1, j + 1] and amplitude_matrix[i, j] > amplitude_matrix[i + 1, j - 1]):
                        g_n[i, j] = amplitude_matrix[i, j]
                    else:
                        g_n[i, j] = 0
            #     - skośny lewy ($d_3$),
            elif quantized_matrix[i, j] == 2:
                    if (amplitude_matrix[i, j] > amplitude_matrix[i - 1, j] and amplitude_matrix[i, j] > amplitude_matrix[i + 1, j]):
                        g_n[i, j] = amplitude_matrix[i, j]
                    else:
                        g_n[i, j] = 0
            #     - skośny prawy ($d_4$).
            elif quantized_matrix[i, j] == 3:
                    if (amplitude_matrix[i, j] > amplitude_matrix[i - 1, j - 1] and amplitude_matrix[i, j] > amplitude_matrix[i + 1, j + 1]):
                        g_n[i, j] = amplitude_matrix[i, j]
                    else:
                        g_n[i, j] = 0
    return g_n

In [ ]:
def canny(image, threshold_h: float, treshold_l: float) -> tuple:
    # 1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
    img_gaussian_blur = cv2.GaussianBlur(image, (3, 3), 0)
    height, width = image.shape
    S1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    S2 = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    # 2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
    g_x = scipy.signal.convolve2d(img_gaussian_blur, S1, mode="same")
    g_y = -scipy.signal.convolve2d(img_gaussian_blur, S2, mode="same")

    # 3. Dalej oblicz amplitudę:
    # $M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
    # $\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
    # Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
    # Wynik jest w radianach.
    amplitude_matrix = np.sqrt(g_x**2 + g_y**2)
    a_d = np.rad2deg(np.arctan2(g_y, g_x))
    a_d[a_d < 0] += 180

    # 4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
    # Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
    # [$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
    # [$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
    # [$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
    # [$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
    # Przy czym należy rozpatrywać tylko 4 kierunki:
    #     - pionowy ($d_1$),
    #     - poziomy ($d_2$),
    #     - skośny lewy ($d_3$),
    #     - skośny prawy ($d_4$).
    quantized_matrix = np.zeros(image.shape)
    for i in range(height):
        for j in range(width):
            if (0 <= a_d[i, j] < 22.5) or (157.5 <= a_d[i, j] <= 180):
                quantized_matrix[i, j] = 0 # poziomy
            elif 22.5 <= a_d[i, j] < 67.5:
                quantized_matrix[i, j] = 1
            elif 67.5 <= a_d[i, j] < 112.5:
                quantized_matrix[i, j] = 2
            elif 112.5 <= a_d[i, j] < 157.5:
                quantized_matrix[i, j] = 3

    # 5 nonmax suppression
    g_n = nonmax(quantized_matrix, amplitude_matrix)

 

    # 6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
    # Stosuje się tutaj tzw. binaryzację z histerezą.
    # Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
    # Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
    # Rezultaty binaryzacji można opisać jako:<br>
    # $g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
    # $g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
    # Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
    # Natomiast na $g_{NL}$ "potencjalne".
    # Często krawędzie "pewne" nie są ciągłe.
    # Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    #     - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
    # W tym celu wykorzystaj listę współrzędnych `[row, col]`.
    # Do pobrania elementu z początku służy metoda `list.pop()`.
    # Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    #     - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    #     - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
    # Jej rozmiar jest równy rozmiarowi obrazu.
    #     - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
    g_nh = g_n >= threshold_h
    g_nl = (g_n >= treshold_l) & (g_n < threshold_h)

    s_g_nh = []
    for i in range(height):
        for j in range(width):
            if g_nh[i, j] == 1:
                s_g_nh.append([i, j])

    visited = np.zeros(image.shape)
    result = np.zeros(image.shape)
    # W tym celu najlepiej sprawdzi się pętla `while`.
    #     - W każdej iteracji pobierz element ze stosu.
    #     - Sprawdź czy dany element został już odwiedzony.
    #     - Jeśli nie został, to:
    #         - Oznacz go jako odwiedzony,
    #         - Oznacz piksel jako krawędź w wyniku,
    #         - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
    #         - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
    #         Można to wykoanać np. pętlą po stworzonym otoczeniu.
    while s_g_nh:
        el = s_g_nh.pop()
        if visited[el[0], el[1]] == 0:
            visited[el[0], el[1]] = 1
            result[el[0], el[1]] = 1
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if g_nl[el[0] + i, el[1] + j] == 1:
                        s_g_nh.append([el[0] + i, el[1] + j])

    return result

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
canny_image_openCV = cv2.Canny(cv2.GaussianBlur(cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE), (3, 3), 0), 100, 200, None, 3, 1)


img_org = canny(cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE), 200, 100)


_, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(canny_image_openCV, 'gray')
ax[0].set_title("Canny OpenCV")
ax[0].axis('off')
ax[1].set_title("Canny ręcznie")
ax[1].imshow(img_org, 'gray')
ax[1].axis('off')
plt.show()